In [11]:
import yahoo_fin.stock_info as si
import datetime
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup


In [33]:
tickers_list = ["aapl", "goog", "amzn", "BAC", "BA"]
tickers_data = {}

for ticker in tickers_list:
    stock = yf.Ticker(ticker)
    
    # convert info() output from dictionary to dataframe
    temp = pd.DataFrame.from_dict(stock.info, orient='index')
    temp.reset_index(inplace=True)
    temp.columns = ['Attribute', 'Value']
    
    # add (ticker, dataframe) to main dictionary
    tickers_data[ticker] = temp
    
combined_data = pd.concat(tickers_data)
combined_data = combined_data.reset_index()

del combined_data['level_1'] # clean up unnecessary column
combined_data.columns = ['Ticker', 'Attribute', 'Value'] # rename columns
# print(combined_data)

In [42]:
# combined_data
#combined_data.drop
records = combined_data.to_dict('records')
#records


In [65]:
def convert_df_to_document(df, date_taken, exclude_columns=[]):
    tickers = df['Ticker'].unique()
    
    for ticker in tickers:
        # Filter the DataFrame for the current ticker
        ticker_df = df[df['Ticker'] == ticker]
        
        # Convert the filtered DataFrame to a dictionary {Attribute: Value} and exclude the columns in exclude_columns
        attributes = ticker_df[~ticker_df['Attribute'].isin(exclude_columns)].set_index('Attribute')['Value'].to_dict()
        
        # Dates
        updates = []
        if ticker_df.get('updates') is not None:
            updates = ticker_df['updates'].tolist()
            
        updates.append(date_taken.strftime('%Y-%m-%d %H:%M:%S'))
                
        # Prepare the document to insert into MongoDB
        document = {'ticker': ticker, 'attributes': attributes, 'updates': updates}
        
        # Insert the document into the collection
        #collection.insert_one(document)
        return document
        

In [81]:
aapl_df = combined_data[combined_data['Ticker'] == 'aapl']
aapl_doc = convert_df_to_document(aapl_df, datetime.datetime.now(), ['companyOfficers', 'uuid'])
aapl_doc

{'ticker': 'aapl',
 'attributes': {'address1': 'One Apple Park Way',
  'city': 'Cupertino',
  'state': 'CA',
  'zip': '95014',
  'country': 'United States',
  'phone': '408 996 1010',
  'website': 'https://www.apple.com',
  'industry': 'Consumer Electronics',
  'industryKey': 'consumer-electronics',
  'industryDisp': 'Consumer Electronics',
  'sector': 'Technology',
  'sectorKey': 'technology',
  'sectorDisp': 'Technology',
  'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital conte

In [40]:
aapl = combined_data[combined_data['Ticker'] == 'aapl']
aapl

,Ticker,Attribute,Value
0,aapl,address1,One Apple Park Way
1,aapl,city,Cupertino
2,aapl,state,CA
3,aapl,zip,95014
4,aapl,country,United States
...,...,...,...
127,aapl,grossMargins,0.45027
128,aapl,ebitdaMargins,0.33733
129,aapl,operatingMargins,0.33764
130,aapl,financialCurrency,USD


In [84]:

aapl = yf.Ticker("AAPL")
aapl_historical_day = aapl.history(period="max", interval="1d")
aapl_historical_hour = aapl.history(period="max", interval="1h")
aapl_historical_week = aapl.history(period="max", interval="1wk")
print(len(aapl_historical_hour))
print(len(aapl_historical_day))
print(len(aapl_historical_week))


AAPL: 1h data not available for startTime=345479400 and endTime=1712673208. The requested range must be within the last 730 days.


0
10920
2262


In [88]:
aapl_historical_day.tail(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-04-02 00:00:00-04:00,169.080002,169.339996,168.229996,168.839996,49329500,0.0,0.0
2024-04-03 00:00:00-04:00,168.789993,170.679993,168.580002,169.649994,47691700,0.0,0.0
2024-04-04 00:00:00-04:00,170.289993,171.919998,168.820007,168.820007,53704400,0.0,0.0
2024-04-05 00:00:00-04:00,169.589996,170.389999,168.949997,169.580002,42055200,0.0,0.0
2024-04-08 00:00:00-04:00,169.029999,169.199997,168.240005,168.449997,37358600,0.0,0.0


In [87]:
aapl_historical_week.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-08 00:00:00-05:00,0.099192,0.099623,0.099192,0.099192,469033600,0.0,0.0
1980-12-15 00:00:00-05:00,0.094448,0.097898,0.087117,0.097467,490134400,0.0,0.0
1980-12-22 00:00:00-05:00,0.102211,0.122912,0.102211,0.122481,187891200,0.0,0.0
1980-12-29 00:00:00-05:00,0.124206,0.124637,0.117737,0.119031,219452800,0.0,0.0
1981-01-05 00:00:00-05:00,0.116875,0.116875,0.104368,0.109974,197904000,0.0,0.0
